### 8-1-4实现字符级的LSTM文本生成

1. 准备数据
首先下载语料，并将其转换为小写。

In [1]:
import keras
import numpy as np


Using TensorFlow backend.


In [2]:
path = keras.utils.get_file('nietzsche.txt',
                           origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 7s 11us/step
Corpus length: 600893


接下来，我们要提取长度为maxlen 的序列（这些序列之间存在部分重叠），对它们进行
one-hot 编码，然后将其打包成形状为(sequences, maxlen, unique_characters) 的三维
Numpy 数组。与此同时，我们还需要准备一个数组y，其中包含对应的目标，即在每一个所提
取的序列之后出现的字符（已进行one-hot 编码）。

- 将字符序列向量化

In [12]:
maxlen = 60 #提取60个字符组成的序列
step = 3 #每3个字符采样一个新序列
sentences = [] #保存所提取的序列
next_chars = [] #保存目标（即下一个字符）

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    #print(i + maxlen)
    next_chars.append(text[i + maxlen])
    
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text))) #语料中唯一字符组成的列表
print('Unique characters:', len(chars))
#一个字典，将唯一字符映射为它在列表chars 中的索引
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorization...')

#将字符one-hot 编码为二进制数组
x = np.zeros((len(sentences),maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i,sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Number of sequences: 200278
Unique characters: 58
Vectorization...


2. 构建网络

这个网络是一个单层LSTM，然后是一个Dense 分类器和对所有可能字符的softmax。但要
注意，循环神经网络并不是序列数据生成的唯一方法，最近已经证明一维卷积神经网络也可以
成功用于序列数据生成。

- 用于预测下一个字符的单层LSTM 模型

In [13]:
from keras import layers

In [14]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95744     
_________________________________________________________________
dense_1 (Dense)              (None, 58)                7482      
Total params: 103,226
Trainable params: 103,226
Non-trainable params: 0
_________________________________________________________________


目标是经过one-hot 编码的，所以训练模型需要使用categorical_crossentropy 作为
损失。

- 模型编译配置

In [18]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

3. 训练语言模型并从中采样

给定一个训练好的模型和一个种子文本片段，我们可以通过重复以下操作来生成新的文本。

(1) 给定目前已生成的文本，从模型中得到下一个字符的概率分布。

(2) 根据某个温度对分布进行重新加权。

(3) 根据重新加权后的分布对下一个字符进行随机采样。

(4) 将新字符添加到文本末尾。

下列代码将对模型得到的原始概率分布进行重新加权，并从中抽取一个字符索引［采样函
数（sampling function）］。

- 给定模型预测，采样下一个字符的函数

In [22]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
    

最后，下面这个循环将反复训练并生成文本。在每轮过后都使用一系列不同的温度值来生成
文本。这样我们可以看到，随着模型收敛，生成的文本如何变化，以及温度对采样策略的影响。

- 文本生成循环

In [23]:
import random
import sys

In [ ]:
for epoch in range(1,60):#将模型训练60轮
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1) #将模型在数据上拟合一次
    
    #随机选择一个文本种子
    start_index = random.randint(0, len(text)- maxlen -1)
    generated_text = text[start_index:start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:#深度一系列不同的采样温度
        print('------ temperature: ', temperature)
        sys.stdout.write(generated_text)
        
        #从种子文本开始，生成400个字符
        for i in range(400):
            #对目前生成的字符进行one-hot 编码
            sampled = np.zeros((1,maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
                
            
            #对下一个字符进行采样
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds,temperature)
            next_char = chars[next_index]
            
            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
        

epoch 1
Epoch 1/1
200278/200278 [==============================] - 106s 528us/step - loss: 1.6206
--- Generating with seed: "hes the narrowing of perspectives, and thus, in
a certain se"
------ temperature:  0.2
hes the narrowing of perspectives, and thus, in
a certain serious the soul and considere the believe the soul is the one of the such the sense of the more the same the soul is the same the soul and such a soul who consequestion of the same the be the have himself the believe the soul and the soul and the success the soul the have have been the have the believe the have have been the success of the senses the fardination of the more the soul the sense of th------ temperature:  0.5
 senses the fardination of the more the soul the sense of the fard of such the be first who disting in the morality he words the more the more a something the sure ither the same the something the fact of the wordd stepe the the to to the depths of the sufficient of the believed in the form of the cons

rat a wholly that id should batfped for leads of cluds, incture imselves. yet arginaty degrous life, ibstawhy this etrainess.

1a
inoverwodoary) a prose-smilly, in subtkests howed only yant; 
no
circumbomse for ishely
with ordenting sin over: i evolu inesuals though fontle: effect parory to his part."--the 
fart that succed the plosothned. good and characeque
and manner--willonusl, were he disby inteary epoch 5
Epoch 1/1
200278/200278 [==============================] - 97s 483us/step - loss: 1.4385
--- Generating with seed: "eakling who
revelled in nothing but anonymous joy and sorrow"
------ temperature:  0.2
eakling who
revelled in nothing but anonymous joy and sorrow and the pational taken in the pression and will to the desire that the condection of the same the art and for the general and all the same the more in the subjective to the condection to the condection of the same the desire the word in the same the discivile, and the desire of the condection of the self-will to the des

awilt intonet of discreaning the philosophers attaend learning end, on thos-writhority may to us e
is responsibility herpari in germanity were trail, about
sive toward the christianity himself and powerful veng herpa, that setures of all of the handfevind through------ temperature:  1.2
ul veng herpa, that setures of all of the handfevind through thinwicys is ngewing howeves sign these cannitoruriwisifal errorling
oncedness ansightyd nature is a tru-frring: all teacher
mung topy may be persoc tatner and teendation people.


1hhame of human to anatiof's, cade? prehable flown lets the exceptione dhe
indeed, such still, shud of simply the sucrain" thro
mlanger senses; peedation of a
fath,ikely, beenout insign at ti strook with
increations. tepoch 9
Epoch 1/1
200278/200278 [==============================] - 98s 487us/step - loss: 1.3895
--- Generating with seed: "ble to stand alone; they succumb to accidents in their isola"
------ temperature:  0.2
ble to stand alone; they succumb to accid

for his fathers whatever that the sense and self-way love and stragated in their the s------ temperature:  1.0
hat the sense and self-way love and stragated in their the subject, happney. this. i to the conceptions to good and has
schopenhauer-"-what is a way been charutes of voluttere the sidor reight to europehous is also
religious then gold always it dopeme for, while
of conscience in europe truth" is domain, and a noble sidity.=--framely taugied.
 though: asavikely
kicce and densebad lover of who and comured
moral mprations to which the livianitys, in his hatic------ temperature:  1.2
omured
moral mprations to which the livianitys, in his hatic most conktage--a coumant-he wurf, and romans musrousy
and .nure this budle; they turgueate
toty
casested, who may modernively, philosophy, he has onvockirnedb, and superstition.wently, closing with venlighbspoping over
slave also apsomentsness.

chass yew utct, pronority; the grefkes, interpretowncoming deceptions.iy these criuther his notd 

d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


...

the compares to the sense of the senses of the more and more man and contrary exceptions and come with one of a man would depths, the spirit is the higher of a man of the precisely
than to a man are really which all the more like the bad agen, the more suffering of developed and second, and suffered to this sense of such are one best ages to be with the one from the more
to be as a soul and deserves of the threately before a test and the come and thinkers ------ temperature:  1.0
es of the threately before a test and the come and thinkers worth as the useful, as and
juagion say: it is the work--airerntions or after leacine of acts beyonds a
germany relantifstion to superby in a need also as
weld anyarity, the fires only one has attained through an situpidity to our "arrangerous more taken the moty and new germans, it is elshiling, and of his soul has to
liwer that maiters rinds and ago mittern
enaucfor to galles to us is distrust p------ temperature:  1.2
rinds and ago mittern
enaucfor

truths. we know well enough how offensive its own the sentiment of the sentiment of the self-consequences of the sentiment of the sentiment of the self-contradictory of the experience in the sentiment of the sentiment of the self-contempt to be precisely the sentiment of the self-education of the self-education of the self-contempt to be present the self-contradiction and precisely the sentiment of the sentiment of the same the self-every s------ temperature:  0.5
 the sentiment of the sentiment of the same the self-every so imbegremanness of a lightenation this as the most earth--and the sentiment is not present still also lives and for him that it is so not ages, and of the same the prevails than the sense of the entire the sentiment of conscience of the elixten, of the sense of the strength of the stronger that it with the matter that it is at the sentiment of the privilegn and so desirate of the self-soul man at------ temperature:  1.0
ent of the privilegn and so desirate of the se

been never is brow and misfortueal, nor sollomiate
far thiling, before promper "ahe specaledness".-
deromatly.--epoch 30
Epoch 1/1
200278/200278 [==============================] - 97s 484us/step - loss: 1.3132
--- Generating with seed: "ses
of conduct as are the ordinary accompaniments of love, o"
------ temperature:  0.2
ses
of conduct as are the ordinary accompaniments of love, of the strange and and still the same the spirit of the same the success of the most conscience of the strength of the strength of the same the most present spirit and thereby with the struggle of the free spirit and problem in the struggle of the same the stronger and stronger and more provers of the same the stronger than the strength and stronger and spirit and stronger and the same the most pro------ temperature:  0.5
d stronger and spirit and stronger and the same the most prooper the bad democratic from the surprised by the conscience, or presposer the most person of the surmencer and the schopenhable fr

could measunedable ; is involving be the sand, woman thought baund
to betogy,
ashed hearon that him whether think hinther, -nudbers of the arrangemens than
lizedly to uso,
zjeitatenc, for in modest cans to
loog and spirits
of "the worf on exercy relagent comcrentous, he has purtion a great firrute
man.jom a bad great rolfers, made, our is other-tomounant, time understand
already alassews his succise
whatever
indreants of valk of explepoch 34
Epoch 1/1
200278/200278 [==============================] - 97s 483us/step - loss: 1.3009
--- Generating with seed: "haps!
but who wishes to concern himself with such dangerous "
------ temperature:  0.2
haps!
but who wishes to concern himself with such dangerous and self-conscience, and the man and conscience, and the sense of the sense of the sense to the conscience, and the senses and the intellectual and conscience of the sindation of the present of the sense that the present stronger and profound, the father and profound to the matters and cons

hobmles belutusting are one call which semilations shall the error; in modeing is a more others than the creature and something a invasent to made which person), and althoughing purpose of herophiness and but adistenesss:--which we do when a plebeture as absolute
time, pressiate but such a development, fastiesism is something cause he is higher for the
were to have not something wise ------ temperature:  1.2
 cause he is higher for the
were to have not something wise themcetertay: one to botiy, to
with it. coups serse, pateess of
"prevail.--what some is their middle--ey pey hetp hahm of fexuled, of puges, but i have him. canton
slave". thucfarisions of xist," he follow by themself to philosopher
effect time has to to willting, in others,
extens to
poetes to be fundamental and "the proviisions, that is been always it see to aspect, however, year
and avery flikeepoch 38
Epoch 1/1
200278/200278 [==============================] - 97s 483us/step - loss: 1.3025
--- Generating with seed: "is 